POC DAX M30 : POC to test different scores with DAX30 data using both 30 min timeframe and 1 day timeframe.
This notebook is the first part : merge dataset, calculate features

In [ ]:
from importlib import reload
import pandas as pd
import numpy as np
from datetime import timedelta,datetime
import ta
import time
from pathlib import Path
import matplotlib.pyplot as plt


In [ ]:
PATH_DATA = "../../../Data/"
PATH_DATA_Quant = "../../../Data/dl_quant/"
PATH_DATA_Yahoo = "../../../Data/dl_yahoo/"

In [ ]:
df_dax_m30=pd.read_csv(PATH_DATA_Quant+"DEUIDXEUR-M30-full.csv",sep=",",parse_dates=True)
df_dax_m30.head()

In [ ]:
df_dax_m30["Date"]=df_dax_m30["Date"].apply(str)
df_dax_m30["Time"]=df_dax_m30["Time"].apply(str)
df_dax_m30.info()

In [ ]:
df_dax_m30["datetime_str"]=df_dax_m30["Date"]+" "+df_dax_m30["Time"]
df_dax_m30["Datetime"]=pd.to_datetime(df_dax_m30["datetime_str"],format="%Y%m%d %X")
df_dax_m30.head()

In [ ]:
df_dax_m30.set_index("Datetime",inplace=True)
df_dax_m30.head()

In [ ]:
df_dax_m30.drop("datetime_str",axis=1,inplace=True)
df_dax_m30.head()

In [ ]:
df_dax_hist=pd.read_csv(PATH_DATA+"dl_yahoo/yf_dax_1d.csv",sep=",",index_col="Date",parse_dates=True)
df_dax_hist.head()

In [ ]:
df_dax_m30["Date"]=pd.to_datetime(df_dax_m30["Date"],format="%Y%m%d")
df_dax_m30.head()

In [ ]:
df_dax_m30_d1=df_dax_m30.join(df_dax_hist, on="Date",how="left",rsuffix="_d1")
df_dax_m30_d1.head()

In [ ]:
df_dax_m30_d1.plot(y=["Close","Close_d1"])

In [ ]:
df_dax_m30_d1.to_csv(PATH_DATA_Quant+"DAX_M30_QDM_D1_YF_base.zip", sep=",")

In [ ]:
df_dax_m30_d1_2=pd.read_csv(PATH_DATA_Quant+"DAX_M30_QDM_D1_YF_base.zip",sep=",",index_col="Datetime",parse_dates=True)
df_dax_m30_d1_2.head()

In [ ]:
df_dax_m30_d1.to_hdf(PATH_DATA_Quant+"DAX_M30_QDM_D1_YF_base.hdf",key="Datetime")

In [ ]:
df_dax_m30_d1.drop(["Dividends","Stock Splits"],axis=1,inplace=True,errors="ignore")
df_dax_m30_d1.head()

In [ ]:
df_dax_m30_d1.to_csv(PATH_DATA_Quant+"DAX_M30_QDM_D1_YF_base.zip", sep=",")


In [ ]:
df_dax_m30_d1 = pd.read_csv(PATH_DATA_Quant+"DAX_M30_QDM_D1_YF_base.zip",sep=",",index_col="Datetime",parse_dates=True)
df_dax_m30_d1.head()

In [ ]:
df_dax_m30_d1['m30_lowest_20']=df_dax_m30_d1['Low'].rolling(20).min().shift(1)
df_dax_m30_d1.head(30)

In [ ]:
df_dax_m30_d1[100:200].plot(y=['Low','m30_lowest_20'])

In [ ]:
df_dax_m30_d1['m30_atr14'] = ta.volatility.AverageTrueRange(
    high=df_dax_m30_d1['High'], low=df_dax_m30_d1['Low'], close=df_dax_m30_d1['Close'], window=14,fillna=False).average_true_range()
df_dax_m30_d1['m30_atr14'] =df_dax_m30_d1['m30_atr14'].shift(1) 
df_dax_m30_d1.head(20)


In [ ]:
df_dax_m30_d1.index=df_dax_m30_d1.index-pd.DateOffset(hours=1)
df_dax_m30_d1.head()

In [ ]:
df_dax_m30_d1['Time']=df_dax_m30_d1.index.strftime('%X')
df_dax_m30_d1.head()

In [ ]:
df_dax_m30_d1['m30_sup_open_jour']=df_dax_m30_d1['Close'].shift(1)>=df_dax_m30_d1['Open_d1']
df_dax_m30_d1.head()

In [ ]:
df_dax_m30_d1['m30_return']=ta.others.daily_return(close=df_dax_m30_d1['Close'],fillna=True)
df_dax_m30_d1.loc[df_dax_m30_d1.index[0],('m30_return')]=0
df_dax_m30_d1['m30_return']=df_dax_m30_d1['m30_return'].shift(1)
df_dax_m30_d1[['Close','m30_return']].head()

In [ ]:
df_dax_m30_d1 = df_dax_m30_d1.round(5)
df_dax_m30_d1[['Close', 'm30_return', 'Close_d1']].head()


In [ ]:
df_dax_m30_d1.round(5).to_csv(PATH_DATA_Quant+"DAX_M30_QDM_D1_YF_TA.zip", sep=",")

In [ ]:
df_dax_m30_d1 = pd.read_csv(PATH_DATA_Quant+"DAX_M30_QDM_D1_YF_TA.zip",sep=",",index_col="Datetime",parse_dates=True)
df_dax_m30_d1.head()

In [ ]:
df_dax_without_d1=df_dax_m30_d1[df_dax_m30_d1['Close_d1'].isnull()]
print(df_dax_without_d1['Date'].unique())
print(df_dax_without_d1.head())

In [ ]:
df_resample_tmp=df_dax_m30_d1['Close'].resample(rule='D').last()
df_dax_resamp_full=pd.DataFrame()
df_dax_resamp_full['Close']=df_resample_tmp

df_dax_resamp_full.head(20)

In [ ]:
df_dax_resamp_full['Open']=df_dax_m30_d1['Open'].resample(rule='D').first()
df_dax_resamp_full['High']=df_dax_m30_d1['High'].resample(rule='D').max()
df_dax_resamp_full['Low']=df_dax_m30_d1['Low'].resample(rule='D').min()

df_dax_resamp_full.head(20)

In [ ]:
df_dax_resamp_full.dropna(inplace=True)
df_dax_resamp_full.index.names=['Date']
df_dax_resamp_full.info()

In [ ]:
df_dax_without_d1=df_dax_m30_d1[df_dax_m30_d1['Close_d1'].isnull()]
df_dax_without_d1.info()


In [ ]:
df_dax_m30_d1.dropna(axis=0,subset=['Close_d1'],inplace=True)
df_dax_m30_d1.info()


In [ ]:
df_dax_m30_d1['m30_rsi14']=ta.momentum.RSIIndicator(df_dax_m30_d1['Close'],window=14).rsi()
df_dax_m30_d1['m30_rsi14']=df_dax_m30_d1['m30_rsi14'].shift(1)
df_dax_m30_d1[['Close', 'm30_rsi14']].head(20)

In [ ]:
df_dax_m30_d1['m30_BB_hband']=ta.volatility.bollinger_hband(df_dax_m30_d1['Close'].shift(1))
df_dax_m30_d1['m30_BB_lband']=ta.volatility.bollinger_lband(df_dax_m30_d1['Close'].shift(1))
df_dax_m30_d1[100:110].head(10)

In [ ]:
df_dax_m30_d1['m30_sma20']=ta.trend.SMAIndicator(close=df_dax_m30_d1['Close'].shift(1),window=20).sma_indicator()
df_dax_m30_d1[['Close','m30_sma20']].head(30)

In [ ]:
df_dax_m30_d1['m30_lowest_50']=df_dax_m30_d1['Low'].rolling(50).min().shift(1)
df_dax_m30_d1['m30_highest_20']=df_dax_m30_d1['High'].rolling(20).min().shift(1)

In [ ]:
df_dax_m30_d1.round(5).to_csv(PATH_DATA_Quant+"DAX_M30_QDM_D1_YF_TA.zip", sep=",")

In [ ]:
df_dax_m30_d1['d1_sma20']=ta.trend.SMAIndicator(close=df_dax_m30_d1['Close_d1'].shift(1),window=20).sma_indicator()
df_dax_m30_d1['d1_sma50']=ta.trend.SMAIndicator(close=df_dax_m30_d1['Close_d1'].shift(1),window=50).sma_indicator()
df_dax_m30_d1['d1_sma200']=ta.trend.SMAIndicator(close=df_dax_m30_d1['Close_d1'].shift(1),window=200).sma_indicator()

In [ ]:
df_dax_m30_d1['d1_rsi14']=ta.momentum.RSIIndicator(df_dax_m30_d1['Close_d1'].shift(1),window=14).rsi()

In [ ]:
df_dax_m30_d1['d1_BB_hband']=ta.volatility.bollinger_hband(df_dax_m30_d1['Close_d1'].shift(1))
df_dax_m30_d1['d1_BB_lband']=ta.volatility.bollinger_lband(df_dax_m30_d1['Close_d1'].shift(1))
df_dax_m30_d1['d1_return']=ta.others.daily_return(close=df_dax_m30_d1['Close_d1'].shift(1))

In [ ]:
df_dax_m30_d1['d1_sma20_volume']=ta.trend.SMAIndicator(close=df_dax_m30_d1['Volume_d1'].shift(1),window=20).sma_indicator()

In [ ]:
df_dax_m30_d1['d1_BB_wband']=ta.volatility.bollinger_wband(df_dax_m30_d1['Close_d1'].shift(1))
df_dax_m30_d1['d1_sma20_wband']=ta.trend.SMAIndicator(close=df_dax_m30_d1['d1_BB_wband'],window=20).sma_indicator()

In [ ]:
df_dax_m30_d1['d1_highest_20']=df_dax_m30_d1['High'].rolling(20).min().shift(1)

In [ ]:
df_dax_m30_d1.drop(['d1_sma20','d1_sma50','d1_sma200','d1_rsi14','d1_BB_hband','d1_BB_lband','d1_return','d1_sma20_volume','d1_BB_wband','d1_sma20_wband','d1_highest_20'],axis=1,inplace=True,errors="ignore")

In [ ]:
df_dax_d1=df_dax_m30_d1.groupby('Date')['Close_d1','Volume_d1'].first()
df_dax_d1=pd.DataFrame(df_dax_d1)
df_dax_d1.head()

In [ ]:
df_dax_d1['d1_sma20']=ta.trend.SMAIndicator(close=df_dax_d1['Close_d1'].shift(1),window=20).sma_indicator()
df_dax_d1['d1_sma50']=ta.trend.SMAIndicator(close=df_dax_d1['Close_d1'].shift(1),window=50).sma_indicator()
df_dax_d1['d1_sma200']=ta.trend.SMAIndicator(close=df_dax_d1['Close_d1'].shift(1),window=200).sma_indicator()
df_dax_d1['d1_rsi14']=ta.momentum.RSIIndicator(df_dax_d1['Close_d1'].shift(1),window=14).rsi()
df_dax_d1['d1_BB_hband']=ta.volatility.bollinger_hband(df_dax_d1['Close_d1'].shift(1))
df_dax_d1['d1_BB_lband']=ta.volatility.bollinger_lband(df_dax_d1['Close_d1'].shift(1))
df_dax_d1['d1_return']=ta.others.daily_return(close=df_dax_d1['Close_d1'].shift(1))
df_dax_d1['d1_sma20_volume']=ta.trend.SMAIndicator(close=df_dax_d1['Volume_d1'].shift(1),window=20).sma_indicator()
df_dax_d1['d1_BB_wband']=ta.volatility.bollinger_wband(df_dax_d1['Close_d1'].shift(1))
df_dax_d1['d1_sma20_wband']=ta.trend.SMAIndicator(close=df_dax_d1['d1_BB_wband'],window=20).sma_indicator()
df_dax_d1['d1_highest_20']=df_dax_d1['Close_d1'].rolling(20).min().shift(1)
df_dax_d1.head()

In [ ]:
df_dax_m30_d1=df_dax_m30_d1.merge(df_dax_d1,on='Date',how='left')

In [ ]:
df_dax_m30_d1["Datetime"]=pd.to_datetime(df_dax_m30_d1["Date"]+" "+df_dax_m30_d1["Time"],format="%Y-%m-%d %X")
df_dax_m30_d1.set_index("Datetime",inplace=True)
df_dax_m30_d1.head()

In [ ]:
df_dax_m30_d1[10000:12000].plot(y=['Close','d1_sma50','d1_sma200','d1_BB_hband','d1_BB_lband'],)

In [ ]:
df_dax_m30_d1.round(5).to_csv(
    PATH_DATA_Quant+"DAX_M30_QDM_D1_YF_TA.zip", sep=",")


In [ ]:
df_dax_m30_d1['m30_market_open']=(df_dax_m30_d1['Time'].shift(1)>="09:00:00") & (df_dax_m30_d1['Time'].shift(1)<="17:30:00")

In [ ]:
df_dax_m30_d1['m30_pos_sma20']=(df_dax_m30_d1['Close'].shift(1)-df_dax_m30_d1['m30_sma20'])/df_dax_m30_d1['m30_sma20']

In [ ]:
df_dax_m30_d1['m30_pos_hband']=(df_dax_m30_d1['Close'].shift(1)-df_dax_m30_d1['m30_BB_hband'])/df_dax_m30_d1['m30_BB_hband']
df_dax_m30_d1['m30_pos_lband']=(df_dax_m30_d1['Close'].shift(1)-df_dax_m30_d1['m30_BB_lband'])/df_dax_m30_d1['m30_BB_lband']

In [ ]:
df_dax_m30_d1['m30_atr_atr14']=((df_dax_m30_d1['High'].shift(1)-df_dax_m30_d1['Low'].shift(1))-df_dax_m30_d1['m30_atr14'])/df_dax_m30_d1['m30_atr14']

In [ ]:
df_dax_m30_d1[['High','Low','m30_atr14','m30_atr_atr14']][100:105]

In [ ]:
df_dax_m30_d1=pd.read_csv(PATH_DATA_Quant+"DAX_M30_QDM_D1_YF_TA.zip",sep=",",index_col="Datetime",parse_dates=True)

In [ ]:
df_dax_m30_d1['m30_is_up']=(df_dax_m30_d1['Close']>df_dax_m30_d1['Open'])
df_dax_m30_d1['m30_is_up']=df_dax_m30_d1['m30_is_up'].shift(1)
df_dax_m30_d1[['Close','Open','m30_is_up']][800:805]

In [ ]:
df_dax_m30_d1['m30_highest_20']=df_dax_m30_d1['Close'].rolling(20).max().shift(1)
df_dax_m30_d1['m30_pos_highest20']=(df_dax_m30_d1['Close'].shift(1)>=df_dax_m30_d1['m30_highest_20'])
df_dax_m30_d1[['Close','m30_highest_20','m30_pos_highest20']][800:805]

In [ ]:
df_dax_m30_d1.rename(columns={"Close_d1_x":"Close_d1","Volume_d1_x":"Volume_d1"},inplace=True)
df_dax_m30_d1[800:805].describe()

In [ ]:
df_dax_m30_d1['d1_highest_20']=df_dax_m30_d1['Close_d1'].rolling(20).max().shift(1)
df_dax_m30_d1[['Close_d1','d1_highest_20']][100:105]

In [ ]:
df_dax_m30_d1['m30_hour_int']=df_dax_m30_d1.index.hour
df_dax_m30_d1[['m30_hour_int']][90:95]

In [ ]:
df_dax_m30_d1['d1_pos_sma20']=(df_dax_m30_d1['Close_d1'].shift(1)-df_dax_m30_d1['d1_sma20'])/df_dax_m30_d1['d1_sma20']
df_dax_m30_d1['d1_pos_sma50']=(df_dax_m30_d1['Close_d1'].shift(1)-df_dax_m30_d1['d1_sma50'])/df_dax_m30_d1['d1_sma50']
df_dax_m30_d1['d1_pos_sma200']=(df_dax_m30_d1['Close_d1'].shift(1)-df_dax_m30_d1['d1_sma200'])/df_dax_m30_d1['d1_sma200']
df_dax_m30_d1['d1_pos_hband']=(df_dax_m30_d1['Close_d1'].shift(1)-df_dax_m30_d1['d1_BB_hband'])/df_dax_m30_d1['d1_BB_hband']
df_dax_m30_d1['d1_pos_lband']=(df_dax_m30_d1['Close_d1'].shift(1)-df_dax_m30_d1['d1_BB_lband'])/df_dax_m30_d1['d1_BB_lband']



In [ ]:
df_dax_m30_d1['d1_BBW_BBw20']=(df_dax_m30_d1['d1_BB_wband']-df_dax_m30_d1['d1_sma20_wband'])/df_dax_m30_d1['d1_sma20_wband']
df_dax_m30_d1['d1_vol_vol20']=(df_dax_m30_d1['Volume_d1']-df_dax_m30_d1['d1_sma20_volume'])/df_dax_m30_d1['d1_sma20_volume']
df_dax_m30_d1['d1_pos_highest20']=(df_dax_m30_d1['Close_d1'].shift(1)>=df_dax_m30_d1['d1_highest_20'])

In [ ]:
df_dax_m30_d1[['Close_d1', 'd1_pos_sma20', 'd1_pos_sma50', 'd1_pos_sma200', 'd1_pos_hband',
               'd1_pos_lband', 'd1_BBW_BBw20', 'd1_vol_vol20', 'd1_pos_highest20']][14800:14805]


In [ ]:
df_dax_m30_d1.drop(columns=['Close_d1_y','Volume_d1_y'],inplace=True)

In [ ]:
df_dax_m30_d1.round(5).to_csv(
    PATH_DATA_Quant+"DAX_M30_QDM_D1_YF_TA_norm.zip", sep=",")


In [ ]:
df_dax_m30_d1.sort_index().loc['2015-01-01':'2017-01-01'].plot(y=['d1_pos_sma20','d1_pos_sma50','d1_pos_sma200','d1_pos_hband','d1_pos_lband'])

In [ ]:
df_dax_m30_d1.sort_index().loc['2015-01-01':'2017-01-01'].plot.hist(y=['d1_pos_sma20','d1_pos_sma50','d1_pos_sma200'],bins=20,alpha=0.5)

In [ ]:
df_dax_m30_d1.sort_index().loc['2015-01-01':'2017-01-01'].plot.hist(y=['d1_pos_hband','d1_pos_lband'],bins=20,alpha=0.5)

In [ ]:
df_dax_m30_d1.sort_index().loc['2014-01-01':'2022-01-01'].plot.hist(y=['d1_vol_vol20_norm'],bins=20,alpha=0.5)

In [ ]:
df_dax_m30_d1['d1_vol_vol20_norm']=np.where(df_dax_m30_d1['d1_vol_vol20']>2,2, df_dax_m30_d1['d1_vol_vol20'])

In [ ]:
df_dax_m30_d1 = pd.read_csv(PATH_DATA_Quant+"DAX_M30_QDM_D1_YF_TA_norm.zip", sep=",",
                            index_col="Datetime", parse_dates=True, keep_default_na=True, low_memory=False)
df_dax_m30_d1.sort_index(inplace = True)
df_dax_m30_d1.info()


In [ ]:
df_dax_m30_d1.astype({'m30_is_up':'bool'}).dtypes
df_dax_m30_d1['m30_is_up'].head()

In [ ]:
df_dax_m30_d1['m30_dist_sl_low50']=(df_dax_m30_d1['Close'].shift(1)-df_dax_m30_d1['m30_lowest_50'])*1.05
df_dax_m30_d1[['Open','m30_lowest_50','m30_dist_sl_low50']][100:105]

In [ ]:
df_dax_m30_d1['m30_dist_sl'] = np.where(df_dax_m30_d1['m30_dist_sl_low50'] < 3 *
                                        df_dax_m30_d1['m30_atr14'], 3*df_dax_m30_d1['m30_atr14'], df_dax_m30_d1['m30_dist_sl_low50'])
df_dax_m30_d1[['m30_dist_sl_low50', 'm30_atr14', 'm30_dist_sl']][100:105]


In [ ]:
df_dax_m30_d1['m30_tp_3sl']=df_dax_m30_d1['Open']+df_dax_m30_d1['m30_dist_sl']*3
df_dax_m30_d1[['Open','m30_dist_sl','m30_tp_3sl']][100:105]

In [ ]:
df_dax_m30_d1.round(5).to_csv(
    PATH_DATA_Quant+"DAX_M30_QDM_D1_YF_TA_norm.zip", sep=",")


In [ ]:
df_dax_m30_d1['m30_date_max_tp']=df_dax_m30_d1.index+timedelta(days=7)
df_dax_m30_d1['m30_date_max_tp']


In [ ]:
df_dax_m30_d1['m30_close_max_7d']=df_dax_m30_d1[::-1]['Close'].rolling('7d', min_periods=20).max()
df_dax_m30_d1[['Close','m30_date_max_tp','m30_close_max_7d']]

In [ ]:
df_dax_m30_d1.drop('m30_close_end',axis=1,errors='ignore')
df_dax_m30_d1.sort_index(inplace = True)
df_dax_m30_d1['m30_close_end']=df_dax_m30_d1['Close'].shift(-7,freq='D')

df_dax_m30_d1['m30_close_end']=df_dax_m30_d1['m30_close_end'].fillna(method='pad')
df_dax_m30_d1[100:105][['Close','m30_date_max_tp','m30_close_max_7d','m30_close_end']]

In [ ]:
df_dax_m30_d1['m30_val_sl']=df_dax_m30_d1['Open']-df_dax_m30_d1['m30_dist_sl']

In [ ]:
df_dax_m30_d1['Datetime_str'] = df_dax_m30_d1['Date']+" "+df_dax_m30_d1['Time']
print(df_dax_m30_d1[77100:77105]['Datetime_str'])

In [ ]:
df_dax_m30_d1['Datetime'] = pd.to_datetime(df_dax_m30_d1['Datetime_str'], format="%Y-%m-%d %X")


In [ ]:
# TOO GREEDY DON'T EXECUTE !!!!
df_dax_m30_d1['m30_exit_2'] = 0
for cur_row in df_dax_m30_d1.itertuples():
    val = 0
    i = 0
    print(f"idx:{cur_row.Index} time:{datetime.now()}" )
    while val==0:
        futur_row = df_dax_m30_d1.shift(-i).loc[cur_row.Index] 
        if futur_row.Low <= cur_row.m30_val_sl:
            val = cur_row.m30_val_sl
        elif futur_row.Close >= cur_row.m30_close_before_end:
            val = cur_row.m30_close_before_end
        elif futur_row.name >= pd.to_datetime(cur_row.m30_date_max_tp,format="%Y-%m-%d %X"):
            val = cur_row.m30_close_end
        else:
            i += 1
            if i > 350:
                val = -1
                print("weird exit")

    df_dax_m30_d1.loc[cur_row.Index]['m30_exit_2'] =val

In [ ]:
df_dax_m30_d1['m30_exit'] = np.nan

for i in range(2*24*5): #2*24*5
    df_shifted = df_dax_m30_d1.shift(-i)
    df_dax_m30_d1['m30_exit_date'] = np.where(np.isnan(df_dax_m30_d1['m30_exit']),df_shifted['Datetime'],df_dax_m30_d1['m30_exit_date'])

    df_dax_m30_d1['m30_exit'] = np.where((np.isnan(df_dax_m30_d1['m30_exit'])) & (df_shifted['Low'] <= df_dax_m30_d1['m30_val_sl']),
                                           df_dax_m30_d1['m30_val_sl'], df_dax_m30_d1['m30_exit'])
    df_dax_m30_d1['m30_exit'] = np.where((np.isnan(df_dax_m30_d1['m30_exit'])) & (df_shifted['Close'] >= df_dax_m30_d1['m30_tp_3sl']),
                                           df_dax_m30_d1['m30_tp_3sl'], df_dax_m30_d1['m30_exit'])
    df_dax_m30_d1['m30_exit'] = np.where((np.isnan(df_dax_m30_d1['m30_exit'] )) & (df_shifted['Datetime'] >= pd.to_datetime(df_dax_m30_d1['m30_date_max_tp'], format="%Y-%m-%d %X")),
                                           df_dax_m30_d1['m30_close_end'], df_dax_m30_d1['m30_exit'])



In [ ]:
df_dax_m30_d1[11300:11310][['m30_val_sl','m30_tp_3sl','m30_close_end','m30_exit_date','m30_exit']]

In [ ]:
df_dax_m30_d1['m30_top_tp_touch']=np.where(df_dax_m30_d1['m30_exit']==df_dax_m30_d1['m30_tp_3sl'],1,0)
df_dax_m30_d1['m30_top_tp_touch'].describe()

In [ ]:
df_dax_m30_d1['m30_top_close_end']=np.where(df_dax_m30_d1['m30_exit']==df_dax_m30_d1['m30_close_end'],1,0)
df_dax_m30_d1['m30_top_close_end'].describe()

In [ ]:
df_dax_m30_d1['m30_hour_norm']=df_dax_m30_d1['m30_hour_int']/24

In [ ]:
df_dax_m30_d1['m30_exit_norm'] = (df_dax_m30_d1['m30_exit']-df_dax_m30_d1['m30_val_sl'])/(
    df_dax_m30_d1['m30_tp_3sl']-df_dax_m30_d1['m30_val_sl'])
df_dax_m30_d1['m30_exit_norm'].describe()	


In [ ]:
df_dax_m30_d1.round(5).to_csv(
    PATH_DATA_Quant+"DAX_M30_QDM_D1_YF_TA_clean.zip", sep=",")


In [ ]:
df_dax_m30_d1 = pd.read_csv(PATH_DATA_Quant+"DAX_M30_QDM_D1_YF_TA_clean.zip", sep=",",
                            index_col="Datetime", parse_dates=True, keep_default_na=True, low_memory=False)
df_dax_m30_d1.sort_index(inplace = True)
df_dax_m30_d1.info()

In [ ]:
df_dax_m30_d1['d1_rsi14']=df_dax_m30_d1['d1_rsi14']/100


In [ ]:
df_dax_m30_d1['m30_perf_exit']=(df_dax_m30_d1['m30_exit']-df_dax_m30_d1['Open'])/df_dax_m30_d1['m30_dist_sl']

In [ ]:
df_dax_m30_d1.sort_index().loc['2014-01-01':'2022-07-01'].plot.hist(y=['m30_perf_exit'],bins=20,alpha=0.5)

In [ ]:
df_dax_m30_d1['m30_perf_exit'].describe()

In [ ]:
df_dax_m30_d1.sort_index().loc['2014-01-01':'2022-01-01'].plot.hist(y=['m30_return'],bins=20,alpha=0.5)